
## OR Study Group: Queueing Theory

### Collaborators: 
* Clare Essex
* Hamish MacGregor
* Rudi Narendran
* Jonathan Teagles
* Emma Tearse

This notebook is designed to run simple queueing theory models using python via mybinder.org



The notebook is structured in XX parts:
* [Set Up](#setup)
* [Theory](#theory1)
* [Simple Queues - M/M/1](#simple)
    * [Case Study 1: XX](#casestudy1)
* [Understanding $\lambda$, $\alpha$ and other variables](#theory2)
* [Complex Queues - M/M/$\infty$](#complex)
    * [Case Study 2: XX](#casestudy2)

## Set Up <a class = "anchor" id = "setup"></a>

We need to install some packages to run this notebook:
* **pandas** - this is a package for *shaping* data
* **numpy** - this is a package with helpful functions for *numerical transformations*
* **matplotlib** - this is a package for *visualisiations*

There are installed using the command **pip install 'package'** and then *imported* into your notebook using **import 'package'**, we can give the package a shortened name as we will need to call it a lot later.

*Note: pip should be run from the command line, to run a shell command from within a notebook cell, you must put a ! in from of the command*

So let's do this for the above packages:

In [ ]:
!pip install pandas
import pandas as pd

In [ ]:
!pip  install numpy
import numpy as np

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt

In [ ]:
!pip install ciw
import ciw

## Theory <a class = "anchor" id = "theory1"></a>

## Simple Queues - M/M/1  <a class = "anchor" id = "simple"></a>

Here we will use the *ciw* package to build a simple M/M/1 queue, such as a queue at a supermarket checkout.
First, we create the _network_ '_N_', which defines the structure of the queueing system.
Functions preceded by _ciw._ are built into the *ciw* package.

In [ ]:
ciw.seed(1) # defines a random seed, ensuring the results are the same on each run

N = ciw.create_network(
    arrival_distributions = [ciw.dists.Exponential(0.2)],
    service_distributions = [ciw.dists.Exponential(0.25)],
    number_of_servers = [1])

This network has three attributes:
* The **arrival distribution**, which we have set to be exponential (Poisson process) with a mean arrival rate $\lambda$ of 0.2 customers per minute (1 every 5 minutes)
* The **service distribution**, which we have also set to be exponential with a mean arrival rate $\mu$ of 0.25 customers per minute (1 every 4 minutes). Since $\lambda < \mu$, the queue should be stable and not grow indefinitely.
* The **number of servers**, which in this case is 1.

We can now simulate the queue by creating and running a *Simulation* object, *Q*:

In [ ]:
Q = ciw.Simulation(N) # a Simulation object for our network N

Q.simulate_until_max_time(1440) # run the simulation Q for 1440 minutes (one day)

The *ciw* package automatically records useful statistics about the simulation. For instance, we can obtain the average time spent waiting in the queue, or the average time to be served:

In [ ]:
recs = Q.get_all_records() # extracts all individual records into the list 'recs'

wait_times = [r.waiting_time for r in recs] # loops through 'recs' extracting waiting times
service_times = [r.service_time for r in recs] # likewise for service times

We can now easily extract the mean waiting time and service time using np.mean():

In [ ]:
np.mean(service_times)

In [ ]:
np.mean(wait_times)

*[Hamish] Idea for an interactive element: get everyone to set a different random seed, then we can use everyone's results to get average values for the waiting times etc*

Let's explore the M/M/1 queue a little further. We can track the state of the system using a 'tracker'. Let's run the same queue three times with different random seeds, and look at the queue length over time.

In [ ]:
for trial in range(3):
    ciw.seed(trial)
    Q2 = ciw.Simulation(N, tracker = ciw.trackers.NodePopulationSubset([0])) #tracks the queue at the service node
    Q2.simulate_until_max_time(480) #simulate for 8 hours
    h = np.array(Q.statetracker.history) # extract an array containing the queue length over time
    h[:, 1] = [i[0] for i in h[:, 1]] # convert tuples to integers
    plt.plot(h[:, 0], h[:, 1])

Try changing the mean arrival rate (at the top of this section) and see how the queue behaviour changes over time  (You will need to re-run any code blocks you change, in order top-to-bottom). What happens if you set the arrival rate higher than the service rate? What if they are equal?

We can automate this experiment as well. Let's try varying the arrival rate, and measuring the average queue length.
Notice that we can't take the time-average queue length when the arrival distribution exceeds the service rate,  since the queue length diverges to infinity over time.
First we set up our trial arrival rates, and lists to store the results:

In [ ]:
test_rates = [0.05, 0.10, 0.15, 0.17, 0.19, 0.21, 0.22, 0.23, 0.24, 0.245, 0.248]
service_rate = 0.25
queue_means = []
queue_var = []

We can now run the simulations:

In [ ]:
for arrival_mean in test_rates:
    N = ciw.create_network(
        arrival_distributions = [ciw.dists.Exponential(arrival_mean)],
        service_distributions = [ciw.dists.Exponential(service_rate)],
        number_of_servers = [1])
    
    Q = ciw.Simulation(N, tracker = ciw.trackers.NodePopulationSubset([0]))
    Q.simulate_until_max_time(1440) # a long simulation gives more accurate results in this experiment
    probs = Q.statetracker.state_probabilities() # a discrete probability distribution for the queue length
    probs = np.array(list(probs.items())) # convert dictionary object to array
    probs[:, 0] = [i[0] for i in probs[:, 0]]
    
    queue_mean = sum(probs[:, 0] * probs[:, 1]) # calculate mean queue length
    queue_means.append(queue_mean)
    queue_var.append(sum(probs[:, 0] * probs[:, 0] * probs[:, 1]) - queue_mean) # calculate queue length variance
    
plt.plot(test_rates, queue_means)

We can see that the average queue length increases rapidly as we approach equal arrival and service rates. The volatility of the queue also increases - long waiting times become much more likely!

(Hamish) We can (and should) compare this with theory - use the simple $r/(1-r)$ for queue length with $r = \lambda/\mu$. I will leave to the theory experts...

## Case Study 1 <a class = "anchor" id = "casestudy1"></a>

## Understanding Variables <a class = "anchor" id = "theory2"></a>

## Complex Queues - M/M/$\infty$ <a class = "anchor" id = "complex"></a>

## Case Study 2 <a class = "anchor" id = "casestudy2"></a>